In [ ]:
#@title ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title データを保存するディレクトリのパスを指定
#@markdown ランタイムに接続しドライブを後、コピーしてくる<br>
#@markdown 後ろの / は追加するので、追記は不要<br>

cwd = "/content/drive/MyDrive/RNA-seq/Outputs/\u5B9F\u9A13\u540D" #@param {type:"string"}
cwd = cwd + "/"

# 必要なモジュールをインポートする
import pandas as pd
import numpy as np
from google.colab import files

# カレントディレクトリの変更
import os
os.makedirs(cwd , exist_ok = True)
os.chdir(cwd)
currentdir = os.getcwd()

In [ ]:
#@title 実験名を定義
exp_name = "\u5B9F\u9A13\u540D" #@param {type:"string"}
exp_name = exp_name + "_"

In [ ]:
#@title 平均行を追加したTPMデータのパスを定義
df_tpm_path =  "" #@param {type:"string"}
df_tpm = pd.read_csv(df_tpm_path , index_col = 0)
df_tpm

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝<br>
GlycoMaple用データの作成

In [ ]:
#@title glycan-related_genes.csv の読み込み
df_glyco_path = "/content/drive/MyDrive/RNAseq/SourceFiles/glycan-related_genes.csv" #@param {type:"string"}
df_glyco = pd.read_csv(df_glyco_path)
# print(df_glyco)

In [ ]:
#@title GlycoMaple用データの抽出
df_glyco_data = pd.merge(df_glyco , df_tpm)
# print(df_glyco_data)
df_glyco_data.to_csv(exp_name + "for_GlycoMaple.csv" , sep = ",")

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝<br>
MetaScape用データの作成

In [ ]:
#@title データの確認
print(list(df_tpm))

In [ ]:
#@title 平均列を比較し、発現変動遺伝子(DEG)を抽出
#@markdown コントロール群の平均列が何列目かを入力
Cont_ave_column = "" #@param {type:"string"}
Cont_num = int(Cont_ave_column) - 1
#@markdown テスト群の平均列が何列目かを入力
Test_ave_column = "" #@param {type:"string"}
Test_num = int(Test_ave_column) - 1

df_ave = df_tpm.iloc[:, [Cont_num , Test_num]]

#@markdown 新しい列「DEG」を追加し(コントロール+1)/(テスト+1)の値を計算<br>
#@markdown 増減の理解しやすさのためにlog2の行も追加<br>
#@markdown 列「DEG」の値に基づいてソート

df_ave.loc[: , "DEG"] = (df_ave.iloc[:,1].values + 1) / (df_ave.iloc[:,0].values + 1)
df_ave.loc[: , "DEG_log2"] = np.log2(df_ave.loc[: , "DEG"])
df_ave.sort_values("DEG" , axis = 0 , ascending=False , inplace = True)
df_ave

In [ ]:
#@title 発現変動遺伝子を決定する
#@markdown 発現変動とする基準値を入力(X倍, 1/X倍を基準とする時のX)
DEG_sta = "2" #@param {type:"string"}
sta = float(DEG_sta)
sta = np.log2(sta)

#@markdown 発現変動遺伝子を増加と減少に分けてそれぞれcsvで保存する

df_DEG_up = df_ave[df_ave["DEG_log2"] >= sta]
df_DEG_down = df_ave[df_ave["DEG_log2"] <= -sta]

df_DEG_up.to_csv(exp_name + "発現" + DEG_sta + "倍増加遺伝子.csv" , sep = ",")
df_DEG_down.to_csv(exp_name + "発現" + DEG_sta + "分の1倍減少遺伝子.csv" , sep = ",")

In [ ]:
#@title 発現変動値を「実験名_DEGavetpm.csv」ファイルに保存
df_ave = df_ave.loc[: , ["DEG" , "DEG_log2"]]
df_ave.reset_index(inplace=True)
df_tpm.reset_index(inplace=True)
df_tpm_deg = pd.merge(df_tpm , df_ave , how = "inner")
# df_tpm_deg = df_tpm_deg.drop("index" , axis = 1)
df_tpm_deg = df_tpm_deg.set_index(["Geneid" , "product"])
df_tpm_deg.to_csv(exp_name + "DEGavetpm.csv" , sep = ",")
df_tpm_deg